In [1]:
import geopandas as gp
import fiona
import pandas as pd
import matplotlib.pyplot as plt
from shapely.geometry import Point, LineString

In [2]:
NY_GEO = gp.read_file("nyc/geo_export_ed1b9dba-ece4-470b-8a1e-2954c9ff29fe.shp", driver='shapefile')
CITI = pd.read_csv('citi_df_sum.csv') #201911-citibike-tripdata.csv
NY_STATION = pd.read_csv('http://web.mta.info/developers/data/nyct/subway/Stations.csv')
NY_LINES = gp.read_file('line/geo_export_1600b818-1305-4ac8-b2d4-6d1ad4428095.shp')
TURNSTILE = pd.read_csv('turnstile/cleaned_turnstile_daily.csv')
BLYN_CITI_END=gp.read_file("BLYN_CITI_END.shp")
#station = pd.read_csv("http://web.mta.info/developers/data/nyct/subway/Stations.csv")

In [3]:
def coord_start(row):
    return Point(row['start station longitude'], row['start station latitude'])

def line_up(row):
    return LineString([[row['start station longitude'], row['start station latitude']], [row['end station longitude'], row['end station latitude']]])

In [4]:
BLYN = NY_GEO[(NY_GEO.boro_name == 'Brooklyn')]

In [5]:
CITI_DF_END = gp.GeoDataFrame(
    CITI, 
    crs = {'init': 'epsg:4326'},
    geometry = gp.points_from_xy(CITI['end station longitude'], CITI['end station latitude']),
    )

In [6]:
STATION_DF = gp.GeoDataFrame(
    NY_STATION, 
    crs = {'init': 'epsg:4326'},
    geometry = gp.points_from_xy(NY_STATION['GTFS Longitude'], NY_STATION['GTFS Latitude']))

In [ ]:
# "clip" a points layer to the boundary of a polygon
#BLYN_CITI = CITI_DF[CITI_DF.geometry.intersects(BLYN.geometry.unary_union)]
#BLYN_CITI_END = CITI_DF_END[CITI_DF_END.geometry.intersects(BLYN.geometry.unary_union)]

In [ ]:
#BLYN_CITI_END.to_file("BLYN_CITI_END.shp")

In [7]:
# "clip" a points layer to the boundary of a polygon
BLYN_STATION = STATION_DF[STATION_DF.geometry.intersects(BLYN.geometry.unary_union)]

In [8]:
# "clip" a points layer to the boundary of a polygon
BLYN_LINES = NY_LINES[NY_LINES.geometry.intersects(BLYN.geometry.unary_union)]

In [9]:
BLYN_STATION['geometry']=BLYN_STATION['geometry'].buffer(.002)

/Users/chen/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
#BLYN_CITI_END = CITI_DF_END[CITI_DF_END.geometry.intersects(BLYN_STATION.geometry.unary_union)]

In [10]:
BLYN_CITI_STATION_MERGED = gp.sjoin(BLYN_CITI_END, BLYN_STATION, how="inner", op='intersects')

In [12]:
BLYN_CITI_STATION_MERGED_SUM = (BLYN_CITI_STATION_MERGED
 .groupby(['Stop Name'])
 ['counts'].sum()
 .reset_index()
 .sort_values("counts", ascending=False))

,Stop Name,counts
4,Atlantic Av - Barclays Ctr,205331
7,Borough Hall,99226
19,DeKalb Av,87538
5,Bedford Av,75588
34,Jay St - MetroTech,69903
55,York St,61753
6,Bergen St,60607
49,Nevins St,60252
48,Nassau Av,44079
31,High St,37772


In [19]:
BLYN_CITI_STATION_MERGED_TOP = BLYN_CITI_STATION_MERGED.merge(BLYN_CITI_STATION_MERGED_SUM.head(10), on=['Stop Name'], how='inner')

In [26]:
BLYN_CITI_STATION_MERGED_TOP.columns = ['tripduration', 'starttime', 'start st_1', 'start station latitude', 'start station longitude',
       'end statio', 'end stat_1', 'end station latitude', 'end station longitude', 'counts_x',
       'geometry', 'index_right', 'Station ID', 'Complex ID', 'GTFS Stop ID',
       'Division', 'Line', 'Stop Name', 'Borough', 'Daytime Routes',
       'Structure', 'GTFS Latitude', 'GTFS Longitude', 'North Direction Label',
       'South Direction Label', 'counts_y']

In [25]:
BLYN_CITI_END

,Unnamed_ 0,start stat,start st_1,start st_2,start st_3,end statio,end stat_1,end stat_2,end stat_3,counts,geometry
0,287376,3423.0,West Drive & Prospect Park West,40.661063,-73.979453,3423.0,West Drive & Prospect Park West,40.661063,-73.979453,5165,POINT (-73.97945 40.66106)
1,289928,3430.0,Richardson St & N Henry St,40.719079,-73.942237,3086.0,Graham Ave & Conselyea St,40.715143,-73.944507,4189,POINT (-73.94451 40.71514)
2,115770,460.0,S 4 St & Wythe Ave,40.712859,-73.965903,3093.0,N 6 St & Bedford Ave,40.717452,-73.958509,3678,POINT (-73.95851 40.71745)
3,127387,481.0,S 3 St & Bedford Ave,40.712605,-73.962644,3093.0,N 6 St & Bedford Ave,40.717452,-73.958509,3448,POINT (-73.95851 40.71745)
4,160549,2002.0,Wythe Ave & Metropolitan Ave,40.716887,-73.963198,3093.0,N 6 St & Bedford Ave,40.717452,-73.958509,3183,POINT (-73.95851 40.71745)
...,...,...,...,...,...,...,...,...,...,...,...
38420,62615,340.0,Madison St & Clinton St,40.712690,-73.987763,3569.0,Franklin Ave & St Marks Ave,40.675832,-73.956168,1,POINT (-73.95617 40.67583)
38421,62662,340.0,Madison St & Clinton St,40.712690,-73.987763,3760.0,Willoughby Ave & Wyckoff Ave,40.705614,-73.921530,1,POINT (-73.92153 40.70561)
38422,62664,340.0,Madison St & Clinton St,40.712690,-73.987763,3763.0,Wyckoff Av & Jefferson St,40.707165,-73.923711,1,POINT (-73.92371 40.70717)
38423,62669,340.0,Madison St & Clinton St,40.712690,-73.987763,3779.0,Hart St & Wyckoff Ave,40.704876,-73.919911,1,POINT (-73.91991 40.70488)


Index(['Unnamed_ 0', 'start stat', 'start st_1', 'start st_2', 'start st_3',
       'end statio', 'end stat_1', 'end stat_2', 'end stat_3', 'counts_x',
       'geometry', 'index_right', 'Station ID', 'Complex ID', 'GTFS Stop ID',
       'Division', 'Line', 'Stop Name', 'Borough', 'Daytime Routes',
       'Structure', 'GTFS Latitude', 'GTFS Longitude', 'North Direction Label',
       'South Direction Label', 'counts_y'],
      dtype='object')

In [27]:
BLYN_CITI_START = gp.GeoDataFrame(
    BLYN_CITI_STATION_MERGED_TOP.copy(),
    crs = {'init': 'epsg:4326'},
    geometry = gp.points_from_xy(BLYN_CITI_STATION_MERGED_TOP['start station longitude'], BLYN_CITI_STATION_MERGED_TOP['start station latitude']),
    )

In [28]:
BLYN_CITI_PATH = gp.GeoDataFrame(
    BLYN_CITI_STATION_MERGED_TOP.copy(),
    crs = {'init': 'epsg:4326'}    )

In [33]:
line_up = BLYN_CITI_PATH.apply(line_up, axis=1)
BLYN_CITI_PATH['geometry_line'] = line_up
BLYN_CITI_PATH.set_geometry('geometry_line')
BLYN_CITI_PATH;

KeyboardInterrupt: 

In [32]:
BLYN_CITI_PATH

,tripduration,starttime,start st_1,start station latitude,start station longitude,end statio,end stat_1,end station latitude,end station longitude,counts_x,...,Stop Name,Borough,Daytime Routes,Structure,GTFS Latitude,GTFS Longitude,North Direction Label,South Direction Label,counts_y,geometry_line
0,115770,460.0,S 4 St & Wythe Ave,40.712859,-73.965903,3093.0,N 6 St & Bedford Ave,40.717452,-73.958509,3678,...,Bedford Av,Bk,L,Subway,40.717304,-73.956872,Manhattan,Canarsie - Rockaway Parkway,75588,"LINESTRING (-73.96590294000001 40.71285887, -7..."
1,127387,481.0,S 3 St & Bedford Ave,40.712605,-73.962644,3093.0,N 6 St & Bedford Ave,40.717452,-73.958509,3448,...,Bedford Av,Bk,L,Subway,40.717304,-73.956872,Manhattan,Canarsie - Rockaway Parkway,75588,"LINESTRING (-73.96264403000001 40.71260486, -7..."
2,160549,2002.0,Wythe Ave & Metropolitan Ave,40.716887,-73.963198,3093.0,N 6 St & Bedford Ave,40.717452,-73.958509,3183,...,Bedford Av,Bk,L,Subway,40.717304,-73.956872,Manhattan,Canarsie - Rockaway Parkway,75588,"LINESTRING (-73.96319799999998 40.716887, -73...."
3,84114,389.0,Broadway & Berry St,40.710446,-73.965251,3093.0,N 6 St & Bedford Ave,40.717452,-73.958509,2117,...,Bedford Av,Bk,L,Subway,40.717304,-73.956872,Manhattan,Canarsie - Rockaway Parkway,75588,"LINESTRING (-73.96525063 40.71044554, -73.9585..."
4,190777,3100.0,Nassau Ave & Newell St,40.724813,-73.947526,3715.0,Driggs Ave & N 9 St,40.718170,-73.955201,2061,...,Bedford Av,Bk,L,Subway,40.717304,-73.956872,Manhattan,Canarsie - Rockaway Parkway,75588,LINESTRING (-73.94752621650694 40.724812564400...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18972,61090,336.0,Sullivan St & Washington Sq,40.730477,-73.999061,3415.0,Prospect Pl & 6 Ave,40.679331,-73.975195,1,...,Bergen St,Bk,2 3,Subway,40.680829,-73.975098,Manhattan,Flatbush - New Lots,60607,"LINESTRING (-73.99906065 40.73047747, -73.9751..."
18973,276521,3394.0,Columbia St & W 9 St,40.676999,-74.006471,3415.0,Prospect Pl & 6 Ave,40.679331,-73.975195,1,...,Bergen St,Bk,2 3,Subway,40.680829,-73.975098,Manhattan,Flatbush - New Lots,60607,"LINESTRING (-74.00647134 40.6769993, -73.97519..."
18974,63049,341.0,Stanton St & Mangin St,40.717821,-73.976289,3415.0,Prospect Pl & 6 Ave,40.679331,-73.975195,1,...,Bergen St,Bk,2 3,Subway,40.680829,-73.975098,Manhattan,Flatbush - New Lots,60607,"LINESTRING (-73.97628939000001 40.71782143, -7..."
18975,276709,3395.0,Henry St & W 9 St,40.676374,-74.003250,3414.0,Bergen St & Flatbush Ave,40.680945,-73.975673,1,...,Bergen St,Bk,2 3,Subway,40.680829,-73.975098,Manhattan,Flatbush - New Lots,60607,"LINESTRING (-74.00324956999999 40.6763744, -73..."


In [ ]:
fig, ax = plt.subplots(figsize=(20, 20))
BLYN.plot(alpha=.5,edgecolor='black', ax=ax)
BLYN_CITI_PATH.plot(alpha=.5,color='orange', ax=ax)
BLYN_CITI_STATION_MERGED.plot(alpha=.5, marker='o', color='red', ax=ax)
BLYN_CITI_START.plot(alpha=.5, marker='o', color='green', ax=ax)
BLYN_CITI_STATION_MERGED_TOP.plot(alpha=.5, marker='o', color='red', ax=ax)
BLYN_STATION.plot(alpha=.5, color='blue', ax=ax)
BLYN_LINES.plot(alpha=.5,color='black', ax=ax)
plt.axis('equal')
ax.set_axis_off()

,Unnamed_ 0,start stat,start st_1,start st_2,start st_3,end statio,end stat_1,end stat_2,end stat_3,counts,geometry
0,287376,3423.0,West Drive & Prospect Park West,40.661063,-73.979453,3423.0,West Drive & Prospect Park West,40.661063,-73.979453,5165,POINT (-73.97945 40.66106)
1,289928,3430.0,Richardson St & N Henry St,40.719079,-73.942237,3086.0,Graham Ave & Conselyea St,40.715143,-73.944507,4189,POINT (-73.94451 40.71514)
2,115770,460.0,S 4 St & Wythe Ave,40.712859,-73.965903,3093.0,N 6 St & Bedford Ave,40.717452,-73.958509,3678,POINT (-73.95851 40.71745)
3,127387,481.0,S 3 St & Bedford Ave,40.712605,-73.962644,3093.0,N 6 St & Bedford Ave,40.717452,-73.958509,3448,POINT (-73.95851 40.71745)
4,160549,2002.0,Wythe Ave & Metropolitan Ave,40.716887,-73.963198,3093.0,N 6 St & Bedford Ave,40.717452,-73.958509,3183,POINT (-73.95851 40.71745)
...,...,...,...,...,...,...,...,...,...,...,...
38420,62615,340.0,Madison St & Clinton St,40.712690,-73.987763,3569.0,Franklin Ave & St Marks Ave,40.675832,-73.956168,1,POINT (-73.95617 40.67583)
38421,62662,340.0,Madison St & Clinton St,40.712690,-73.987763,3760.0,Willoughby Ave & Wyckoff Ave,40.705614,-73.921530,1,POINT (-73.92153 40.70561)
38422,62664,340.0,Madison St & Clinton St,40.712690,-73.987763,3763.0,Wyckoff Av & Jefferson St,40.707165,-73.923711,1,POINT (-73.92371 40.70717)
38423,62669,340.0,Madison St & Clinton St,40.712690,-73.987763,3779.0,Hart St & Wyckoff Ave,40.704876,-73.919911,1,POINT (-73.91991 40.70488)
